In [80]:
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
from sklearn import preprocessing
import math
from datetime import datetime

In [81]:
def handleNaNTrans(transactions_df,  products_df):
    for i in transactions_df[transactions_df['day'].isnull()].index:
        transactions_df['day'][i] = transactions_df['day'][i-1]
    
    for i in transactions_df[transactions_df['time'].isnull()].index:
        transactions_df['time'][i] = transactions_df['time'][i-1]
    
    for i in transactions_df[transactions_df['product_id'].isnull()].index:
        for a in range(len(products_df)):
            if transactions_df['description'][i] == products_df['description'][a]:
                transactions_df['product_id'][i] = products_df['product_id'][a]
                
    for i in transactions_df[transactions_df['description'].isnull()].index:
        for a in range(len(products_df)):
            if transactions_df['product_id'][i] == products_df['product_id'][a]:
                transactions_df['description'][i] = products_df['description'][a]

    return transactions_df

In [82]:
data_inventory = pd.read_csv("./data/inventory.csv")
data_products = pd.read_csv("./data/products.csv")
data_promotions = pd.read_csv("./data/promotions.csv")
data_simulprom = handleNaNTrans(pd.read_csv(r".\transactions_months_simulprom_subst.csv"), data_products)

C:\Users\Windows10\anaconda3\envs\tf-keras-gpu\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [65]:
data_transactions[data_transactions['product_id'].isnull()]

,day,time,customer,bank acount,category,product_id,description,size,std_sales_price,purchase_price,bio,basic
65737,10/3/2018,18:59:42,3894.0,65327490.0,bread,NaN,NaN,1st,0.99,0.99,0.0,0.0


In [66]:
data_holiday = pd.read_excel(r".\feestdagen_nl_b_2018.xlsx")

C:\Users\Windows10\anaconda3\envs\tf-keras-gpu\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. As a result, the openpyxl engine will be used if it is installed and the engine argument is not specified. Install openpyxl instead.
  """Entry point for launching an IPython kernel.


In [67]:
def parse_holiday(data_hol):
    data_hol = data_hol[data_holiday['Land'] == 'NL']
    data_hol['day'] = 0
    data_hol['day2'] = 0
    data_hol['day_of_year'] = 0
    
    for i in range(len(data_hol.index)):
        data_hol['day'][i] = data_hol['Datum'][i].strftime('%Y-%m-%d')
        data_hol['day'][i] = datetime.strptime(data_hol['day'][i], '%Y-%m-%d').strftime('%d-%m-%Y')
        data_hol['day2'][i] = datetime.strptime(data_hol['day'][i], '%d-%m-%Y').strftime('%d/%m/%Y')
        data_hol['day_of_year'][i] = data_hol['Datum'][i].timetuple().tm_yday
        
    return data_hol
    
    

In [68]:
data_holiday = parse_holiday(data_holiday)
data_holiday

C:\Users\Windows10\anaconda3\envs\tf-keras-gpu\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Windows10\anaconda3\envs\tf-keras-gpu\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\Windows10\anaconda3\envs\tf-keras-gpu\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value

,Datum,Omschrijving,Land,day,day2,day_of_year
0,2018-01-01,Nieuwjaarsdag,NL,01-01-2018,01/01/2018,1
1,2018-03-30,Goede vrijdag,NL,30-03-2018,30/03/2018,89
2,2018-04-01,1e Paasdag,NL,01-04-2018,01/04/2018,91
3,2018-04-02,2e Paasdag,NL,02-04-2018,02/04/2018,92
4,2018-04-27,Koningsdag,NL,27-04-2018,27/04/2018,117
5,2018-05-05,Bevrijdingsdag,NL,05-05-2018,05/05/2018,125
6,2015-05-10,Hemelvaartsdag,NL,10-05-2015,10/05/2015,130
7,2018-05-20,1e Pinksterdag,NL,20-05-2018,20/05/2018,140
8,2018-05-21,2e Pinksterdag,NL,21-05-2018,21/05/2018,141
9,2018-12-25,1e Kerstdag,NL,25-12-2018,25/12/2018,359


In [69]:
def add_holidays(data_prom, data_holiday):
    data_new = data_prom
    data_prom['holiday'] = 0
    #nr_days = len(data_prom['day'].unique())+2
    
    for day in range(1, len(data_prom['day'].unique())+1):
        unique_days = data_prom['day'].unique()
        date = unique_days[day-1]
        
        if day in data_holiday['day_of_year']:
            idx_data = data_prom[data_new['day'] == date]
            idx = idx_data.index
            data_prom.loc[idx, 'holiday'] = 1
            
    return data_prom
    
    

In [70]:
def daily_sales(total_data, product):
    
    product_data = total_data[total_data['description']==product]
    category = product_data['category'].iloc[0]
    
    size_nan = product_data['size'].unique()
    size_nan = [x for x in size_nan if x == x]
    
    new_df = pd.DataFrame()
    total_days_size = len(total_data['day'].unique()) * len(size_nan)
    rows = [x for x in range(total_days_size)]
    new_df['idx'] = rows
    
    new_df['week'] = 0
    new_df['daily_sales'] = 0
    new_df['size'] = 0
    new_df['purchase_price'] = 0
    new_df['month'] = 0
    new_df['simul_promotions'] = 0
    new_df['substitute_promotions'] = 0
    new_df['day'] = 0
    new_df['holiday'] = 0
    new_df['category'] = category
    new_df['product'] = product
    
    week = 1
    index = 0
    day_nr = 0
    
    for day in total_data['day'].unique():
        daily_sales = product_data[product_data['day'] == day]
        data_day = total_data[total_data['day'] == day]
        
        size_int = 0
        for size in size_nan:
            relevant_sales = daily_sales[daily_sales['size'] == size]
            indexors = relevant_sales.index
            
            new_df.loc[index, 'daily_sales'] = len(relevant_sales)
            new_df.loc[index, 'size'] = size_int
            new_df.loc[index, 'week'] = week
            new_df.loc[index, 'day'] = day 
            new_df.loc[index, 'month'] = total_data[total_data['day'] == day]['month'].iloc[0]
            
            try:
                new_df.loc[index, 'simul_promotions'] = product_data[product_data['day'] == day]['simul_promotions'].iloc[0]
            except:
                new_df.loc[index, 'simul_promotions'] = 0
            
            try: 
                new_df.loc[index, 'substitute_promotions'] = relevant_sales['substitute_in_promo'].max()
            except:
                new_df.loc[index, 'substitute_promotions'] = 0
            
            try:
                new_df.loc[index, 'purchase_price'] = relevant_sales['purchase_price'].iloc[0]
            except: 
                new_df.loc[index, 'purchase_price'] = product_data['purchase_price'].max()
                
            try:
                new_df.loc[index, 'holiday'] = data_day['holiday'].iloc[0]
            except:
                new_df.loc[index, 'holiday'] = 0
            
            index += 1
            size_int += 1
        
        day_nr += 1
        week = (day_nr // 7) + 1
        
    
    return new_df

In [71]:
def daily_to_restock(data_sales, data_restock):
    

    month_list = []
    price_list = []
    size_list = []
    sales_list = []
    simulprom_list = []
    days_inbetween_list = []
    subst_list = []
    holiday_list = []
    category = data_sales['category'].iloc[0]
    product = data_sales['product'].iloc[0]
    
    
    for size in data_sales['size'].unique():
        rel_data = data_sales[data_sales['size'] == size]
        rel_data.reset_index()
        day_before = 0
        
        for day in data_inventory['day'].unique()[1:]:
            rel_sales = rel_data[day_before : day]
            
            sales = sum(rel_sales['daily_sales'])
            month = rel_sales['month'].iloc[0]
            price = rel_sales['purchase_price'].iloc[0]
            simulprom = rel_sales['simul_promotions'].iloc[0]
            subst = rel_sales['substitute_promotions'].max()
            holiday = rel_sales['holiday'].max()
            days_inbetween = (day - day_before)
            
            holiday_list.append(holiday)
            sales_list.append(sales)
            size_list.append(size)
            month_list.append(month)
            price_list.append(price)
            simulprom_list.append(simulprom)
            subst_list.append(subst)
            days_inbetween_list.append(days_inbetween)
            
            day_before = day
            
    raw_data = {'month':month_list,  'size':size_list, 'price':price_list, 'sales':sales_list,
                'simul_promotions': simulprom_list, 'days_inbetween': days_inbetween_list, 'substitute_promotions': subst_list,
                'holiday':holiday_list}
    new_data = pd.DataFrame(raw_data)
    new_data['category'] = category
    new_data['product'] = product
    return new_data 

In [212]:
def forest_variables(total_data, productEncoder):
    prices = total_data['price']
    sales = total_data['sales']
    sizes = total_data['size']
    months = total_data['month']
    simulprom = total_data['simul_promotions']
    subst_prom = total_data['substitute_promotions']
    days_inbetween = total_data['days_inbetween']
    holidays = total_data['holiday']
    products = np.array(total_data['product']).reshape(-1, 1)
    products = productEncoder.transform(products).toarray().tolist()
    
    zipped = zip(prices, sizes, days_inbetween, months, subst_prom, simulprom, holidays)
    x_variables = [list(a) for a in zipped]
    x_variables = list(map(list.__add__, x_variables, products))
    y_variables = sales
    
    return x_variables, y_variables

In [73]:
def RMSE(bos, X_test, y_test):
    predictions = bos.predict(X_test)
    rounded_predictions = [round(num) for num in predictions]
    SE = 0 
    
    for i in range(len(y_test)):
        squared_error = ((rounded_predictions[i] - y_test.iloc[i])**2)
        SE += squared_error
        
    MSE = SE/ len(y_test)
    MSE = int(MSE)
    
    RMSE = math.sqrt(MSE)
    
    return RMSE
    

## run preprocessing

In [83]:
data_simulprom_hol = add_holidays(data_simulprom, data_holiday)

In [85]:
data_simulprom_hol_relevant = data_simulprom_hol[data_simulprom_hol['category'] == 'vegetable']
vegetable_products = data_simulprom_hol_relevant['description'].unique().tolist()

In [86]:
vegetable_products

['Boerenkool gesneden',
 'Buitenbeentjes komkommer',
 'Nasi bami Vegetable',
 'Basis worteltjes fijn',
 'Courgette',
 'SnoepVegetable tomaat',
 'Geschrapte worteltjes',
 'Snijbonen',
 'Buitenbeentjes puntpaprika',
 'Buitenbeentjes paprika',
 'Basis sperziebonen',
 'Komkommer',
 'Spruiten',
 'Wortelen',
 'Basis Maiskorrels',
 'SnoepVegetable worteltjes',
 'Basis Sperziebonen',
 'Verse zuurkool',
 'Basis Rode kool met appel',
 'Biologische rode kool',
 'Biologische pompoen',
 'Paprika',
 'Biologische cherry tomaten',
 'Basis erwten zeer fijn',
 'Biologische prei',
 'Biologische Bloemkool',
 'Biologische courgette',
 'Witlof',
 'Paprika Mix',
 'Biologische brocolli',
 'Biologische knoflook',
 'Rucola',
 'WokVegetable thais',
 'Pompoen stukjes',
 'Biologische kikkererwten']

In [123]:
for idx, vegetable in enumerate(vegetable_products):
    if idx == 0:
        data_daily_complete = daily_sales(data_simulprom_hol_relevant, vegetable)
        restocked_data_complete = daily_to_restock(data_daily_complete, data_inventory)
    else:
        data_daily = daily_sales(data_simulprom_hol_relevant, vegetable)
        restocked_data = daily_to_restock(data_daily, data_inventory)
        restocked_data_complete = restocked_data_complete.append(restocked_data)

array(['Boerenkool gesneden'], dtype=object)

In [139]:
restocked_data_complete['product'].unique()

array(['Boerenkool gesneden', 'Buitenbeentjes komkommer',
       'Nasi bami Vegetable', 'Basis worteltjes fijn', 'Courgette',
       'SnoepVegetable tomaat', 'Geschrapte worteltjes', 'Snijbonen',
       'Buitenbeentjes puntpaprika', 'Buitenbeentjes paprika',
       'Basis sperziebonen', 'Komkommer', 'Spruiten', 'Wortelen',
       'Basis Maiskorrels', 'SnoepVegetable worteltjes',
       'Basis Sperziebonen', 'Verse zuurkool',
       'Basis Rode kool met appel', 'Biologische rode kool',
       'Biologische pompoen', 'Paprika', 'Biologische cherry tomaten',
       'Basis erwten zeer fijn', 'Biologische prei',
       'Biologische Bloemkool', 'Biologische courgette', 'Witlof',
       'Paprika Mix', 'Biologische brocolli', 'Biologische knoflook',
       'Rucola', 'WokVegetable thais', 'Pompoen stukjes',
       'Biologische kikkererwten'], dtype=object)

In [271]:
restocked_data_complete.dropna(subset = ["price"], inplace=True)

In [294]:
train_df, test_df = train_test_split(restocked_data_complete, test_size = 0.25, stratify = restocked_data_complete[['month', 'size', 'product', 'days_inbetween']])

In [344]:
train_df.to_csv('./test_train_data/train.csv')
test_df.to_csv('./test_train_data/test.csv')

In [295]:
from sklearn.preprocessing import OneHotEncoder

unique_products = restocked_data_complete['product'].unique().reshape(-1, 1)

productEncoder = OneHotEncoder(dtype=int).fit(unique_products)
productEncoder = productEncoder.fit(unique_products)

X_train, y_train = forest_variables(train_df, productEncoder)
X_test, y_test = forest_variables(test_df, productEncoder)

In [296]:
restocked_data_complete

,index,month,size,price,sales,simul_promotions,days_inbetween,substitute_promotions,holiday,category,product
0,0,1,0,1.29,24,0,3,0.0,1,vegetable,Boerenkool gesneden
1,1,1,0,1.29,43,0,4,0.0,1,vegetable,Boerenkool gesneden
2,2,1,0,1.29,24,0,3,0.0,1,vegetable,Boerenkool gesneden
3,3,1,0,1.29,48,0,4,0.0,1,vegetable,Boerenkool gesneden
4,4,1,0,1.29,36,0,3,0.0,0,vegetable,Boerenkool gesneden
...,...,...,...,...,...,...,...,...,...,...,...
3600,98,12,0,0.92,12,0,3,0.0,0,vegetable,Biologische kikkererwten
3601,99,12,0,0.92,17,0,4,0.0,0,vegetable,Biologische kikkererwten
3602,100,12,0,0.92,5,0,3,0.0,0,vegetable,Biologische kikkererwten
3603,101,12,0,0.92,3,0,4,0.0,0,vegetable,Biologische kikkererwten


## run randomforest

In [338]:
regression_max_features = math.ceil(train_df.columns.shape[0]/3)
rfr = RandomForestRegressor(n_estimators = 64, min_samples_split = 0.01, min_samples_leaf = 0.01, max_features = 'sqrt', max_depth = 64, bootstrap = True)
rfr.fit(X_train, y_train)

RandomForestRegressor(max_depth=64, max_features='sqrt', min_samples_leaf=0.01,
                      min_samples_split=0.01, n_estimators=64)

In [339]:
y_pred = rfr.predict(X_test)

In [340]:
from sklearn import metrics
rmse = np.sqrt(metrics.mean_squared_error(y_test, y_pred))

In [341]:
rmse

8.59169615047823

In [ ]:
{'n_estimators': 250,
 'min_samples_split': 0.02,
 'min_samples_leaf': 0.02,
 'max_features': 'auto',
 'max_depth': 60,
 'bootstrap': True}

In [343]:
import joblib
# save
joblib.dump(rfr, "./random_forest/best_model.joblib")

['./random_forest/best_model.joblib']

## parameter_tuning

In [307]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn import decomposition, datasets
from sklearn.pipeline import Pipeline
from sklearn import tree

In [332]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 50, stop = 500, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt', regression_max_features]
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(20, 200, num = 10)]
# Minimum number of samples required to split a node
min_samples_split = [0.02, 0.05, 0.01]
# Minimum number of samples required at each leaf node
min_samples_leaf = [0.02, 0.05, 0.01]
# Method of selecting samples for training each tree
bootstrap = [True]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

print(random_grid)

{'n_estimators': [50, 100, 150, 200, 250, 300, 350, 400, 450, 500], 'max_features': ['auto', 'sqrt', 4], 'max_depth': [20, 40, 60, 80, 100, 120, 140, 160, 180, 200], 'min_samples_split': [0.02, 0.05, 0.01], 'min_samples_leaf': [0.02, 0.05, 0.01], 'bootstrap': [True]}


In [333]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
#rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 150, cv = 3, verbose=10, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 150 candidates, totalling 450 fits


RandomizedSearchCV(cv=3, estimator=RandomForestRegressor(), n_iter=150,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True],
                                        'max_depth': [20, 40, 60, 80, 100, 120,
                                                      140, 160, 180, 200],
                                        'max_features': ['auto', 'sqrt', 4],
                                        'min_samples_leaf': [0.02, 0.05, 0.01],
                                        'min_samples_split': [0.02, 0.05, 0.01],
                                        'n_estimators': [50, 100, 150, 200, 250,
                                                         300, 350, 400, 450,
                                                         500]},
                   random_state=42, verbose=10)

In [334]:
rf_random.best_params_

{'n_estimators': 350,
 'min_samples_split': 0.01,
 'min_samples_leaf': 0.01,
 'max_features': 'sqrt',
 'max_depth': 80,
 'bootstrap': True}

In [328]:
# Number of trees in random forest
n_estimators = [250, 64]
# Number of features to consider at every split
max_features = ['auto']
# Maximum number of levels in tree
max_depth = [60, 50]
# Minimum number of samples required to split a node
min_samples_split = [0.02]
# Minimum number of samples required at each leaf node
min_samples_leaf = [0.02]
# Method of selecting samples for training each tree
bootstrap = [True]
# Create the random grid
grid_best_param = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [335]:
try:
    for key, value in grid_best_param.items():
        best_param = rf_random.best_params_[key]
        if best_param not in value:
            value.append(rf_random.best_params_[key])
except:
    grid_best_param = {key: [value] for key, value in rf_random.best_params_.items()}

grid_best_param

{'n_estimators': [250, 64, 350],
 'max_features': ['auto', 'sqrt'],
 'max_depth': [60, 50, 80],
 'min_samples_split': [0.02, 0.01],
 'min_samples_leaf': [0.02, 0.01],
 'bootstrap': [True]}

In [336]:
rf = RandomForestRegressor()
rf_grid = GridSearchCV(estimator = rf, param_grid = grid_best_param, cv = 3, n_jobs = -1, verbose = 10)
rf_grid.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'bootstrap': [True], 'max_depth': [60, 50, 80],
                         'max_features': ['auto', 'sqrt'],
                         'min_samples_leaf': [0.02, 0.01],
                         'min_samples_split': [0.02, 0.01],
                         'n_estimators': [250, 64, 350]})

In [337]:
rf_grid.best_params_

{'bootstrap': True,
 'max_depth': 60,
 'max_features': 'sqrt',
 'min_samples_leaf': 0.01,
 'min_samples_split': 0.01,
 'n_estimators': 64}

In [ ]:
def tree_parameters(X, y):
    X = np.array(X)
    
    std_slc = StandardScaler()
    pca = decomposition.PCA()
    dec_tree = DecisionTreeClassifier()
    
    pipe = Pipeline(steps=[('std_slc', std_slc),
                           ('pca', pca),
                           ('dec_tree', dec_tree)])
    
    n_components = list(range(1,X.shape[1]+1,1))
    
    criterion = ['gini', 'entropy']
    max_depth = [2,4,6,8,10,12,14,16]
    
    parameters = dict(pca__n_components=n_components,
                      dec_tree__criterion=criterion,
                      dec_tree__max_depth=max_depth)
    
    clf_GS = GridSearchCV(pipe, parameters)
    clf_GS.fit(X, y)
    
    print('Best Criterion:', clf_GS.best_estimator_.get_params()['dec_tree__criterion'])
    print('Best max_depth:', clf_GS.best_estimator_.get_params()['dec_tree__max_depth'])
    print('Best Number Of Components:', clf_GS.best_estimator_.get_params()['pca__n_components'])
    print(); print(clf_GS.best_estimator_.get_params()['dec_tree'])

In [ ]:
tree_parameters(X_train, y_train)

In [974]:
def find_general_RMSE(total_data):
    RMSE_list = []
    
    for i in range(20):
        train_df, test_df = train_test_split(restocked_data, test_size = 0.25, stratify = restocked_data[['size', 'month', 'days_inbetween']])
        X_train, y_train = forest_variables(train_df)
        X_test, y_test = forest_variables(test_df)
        
        bos_reg = RandomForestRegressor()
        bos_reg.fit(X_train, y_train)
        
        RMSE1 = RMSE(bos_reg, X_test, y_test)
        RMSE_list.append(RMSE1)
        mean = sum(RMSE_list)/len(RMSE_list)
     
    return RMSE_list, mean


    

In [980]:
find_general_RMSE(restocked_data)

C:\Users\20193727\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\20193727\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\20193727\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\20193727\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 t

([5.830951894845301,
  5.744562646538029,
  6.557438524302,
  5.656854249492381,
  5.477225575051661,
  5.744562646538029,
  6.0,
  4.69041575982343,
  5.0990195135927845,
  6.0,
  5.196152422706632,
  4.795831523312719,
  6.0,
  5.0990195135927845,
  5.196152422706632,
  4.898979485566356,
  4.58257569495584,
  5.916079783099616,
  6.244997998398398,
  4.358898943540674],
 5.454485929903163)

In [804]:
y_test

69     10
129    11
181    22
72     12
153    21
34     17
43     18
191    12
174    10
102     7
173    10
8       8
15     13
157    13
186    15
146    14
109    10
110    15
56     10
47     35
134    19
99     28
38     11
91     15
171    11
113     7
71     29
116     9
127     7
4      12
17     12
202    18
196    10
64     17
199     7
57     13
79     14
27     20
140    20
22     13
126    16
148    19
141     7
10     13
84     12
94     17
187    16
46     24
89     20
156    17
166    14
1      14
Name: sales, dtype: int64

In [915]:
for i in range(1, 3):
    print(i)

1
2
